In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
df1 = pd.read_csv('../data/meta_allstreams.csv', converters={'feature': eval,'category': eval, 'also_buy': eval,'also_view': eval, 'details': eval})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29471 entries, 0 to 29470
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     29471 non-null  object
 1   description  27437 non-null  object
 2   title        29470 non-null  object
 3   also_buy     29471 non-null  object
 4   brand        25969 non-null  object
 5   feature      29471 non-null  object
 6   rank         29471 non-null  object
 7   also_view    29471 non-null  object
 8   asin         29471 non-null  object
 9   details      29471 non-null  object
dtypes: object(10)
memory usage: 2.2+ MB


In [ ]:
df1.head()

In [ ]:
df1['details'].value_counts().head(20)

In [ ]:
df1['rank'].value_counts()

In [ ]:
df1['category'][999]

In [ ]:
df1['details'].apply(type).unique()

In [ ]:
df_check_genres = df1[df1['details'].apply(lambda x: 'Language' in x)]
df_check_genres['details'].value_counts()

### Note: 
- Found that 'feature' had 27,736 identical entries and not going to be useful 
- May want to look at also_buy and also_view after the recommendation system is built to see if it's similar to Amazons algorithm.

In [ ]:
df = df1.drop(columns = ['also_buy', 'also_view', 'feature'], axis=1)
#df = df1.drop(columns = ['also_buy', 'also_view', 'details'], axis=1)
df.rename(columns={'category': 'genre', 'brand': 'starring'}, inplace=True)

In [ ]:
def remove_genre_from_list(lst):
    return [genre for genre in lst if genre not in remove_genre]

In [ ]:
df['genre'] = [x[1:] if len(x) > 1 and x[0] == 'Movies & TV' else x for x in df['genre']]
df = df[~df['genre'].apply(lambda x: 'Exercise & Fitness' in x)]
df.loc[df['genre'].apply(lambda x: isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International'), 'genre'] = df['genre'].apply(lambda x: [x[0] + ' ' + x[2]] if len(x) > 2 else x)
df['genre'] = df['genre'].apply(lambda x: x[:1] + x[2:] if isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International' and len(x) > 2 else x)


remove_genre = ['Genre for Featured Categories', 'Boxed Sets', 'All Titles', 
                'Independently Distributed', 'All Sony Pictures Titles', 'Studio Specials',
               'By Original Language', 'All BBC Titles', 'General',
               '20th Century Fox Home Entertainment', 'Family Features',
               'Lionsgate Home Entertainment', 'Fully Loaded DVDs', 'Blu-ray',
               'Holidays & Seasonal', 'All HBO Titles', 'Musicals & Performing Arts',
               'Paramount Home Entertainment', 'Music Artists', 'Walt Disney Studios Home Entertainment',
               'All', 'Universal Studios Home Entertainment'] 



df['genre'] = df['genre'].apply(lambda x: remove_genre_from_list(x) if isinstance(x, list) else x)
df['genre'] = df['genre'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [ ]:
df['rank'] = df['rank'].str.extract(r'([\d,]+)', expand=False).str.replace(',', '').replace('', np.nan).astype(float).astype('Int64')

In [ ]:
df.replace({'description': "Quick Shipping !!! New And Sealed !!! This Disc WILL NOT play on standard US DVD player. A multi-region PAL/NTSC DVD player is request to view it in USA/Canada. Please Review Description."}, np.nan, inplace=True)
df.replace({'description': "Original Reliance DVD"}, np.nan, inplace=True)
df.replace({'description': "Like New Condition! Quick Shipping (Within 24-48hrs). DVD Case & Artwork In Excellent Condition! Previous Rental Some items include Azura Disc Scratch Protection. Digital Copy not available!"}, np.nan, inplace=True)
df.replace({'description': "Original Yashraj DVD"}, np.nan, inplace=True)
df.replace({'description': "Original Eros DVD"}, np.nan, inplace=True)
df.replace({'description': "<i>When sold by Amazon.com, this product will be manufactured on demand using DVD-R recordable media. Amazon.com's standard return policy will apply.</i>"}, np.nan, inplace=True)
df.replace({'description': "NOTICE: Polish Release, cover may contain Polish text/markings. The disk has English audio."}, np.nan, inplace=True)

In [ ]:
df.replace({'description': "DVD"}, np.nan, inplace=True)
df.replace({'description': "dvd"}, np.nan, inplace=True)
df.replace({'description': "BRAY"}, np.nan, inplace=True)

In [ ]:
convert = ['.', '\n', '-', '--', 'Na', 'BRIDGESTONE MULTIMEDIA']
df['starring'] = df['starring'].apply(lambda x: 'Various' if isinstance(x, str) and (x in convert or '\n' in x) else x)
df['starring'].fillna('Various', inplace=True)

In [ ]:
df['details'] = df['details'].apply(lambda x:
                      {k:v for k,v in x.items() if not str(k).strip().startswith('ASIN:')})
df['details'] = [{'Language:': 'English'} if not d else d for d in df['details']]

In [ ]:
df['english'] = df['details'].apply(lambda x: x.get('Language:', ''))
df['english'] = df['details'].apply(lambda x: x.get('Language:', '').split(' (')[0])
df['english'] = df['english'].apply(lambda x: x.split(", "))
df['english'] = df['english'].apply(lambda x: 1 if any('English' in sublist for sublist in x) else 0)
df.drop(columns = 'details', axis=1, inplace=True)

In [ ]:
df.dropna(subset=['description', 'rank'], inplace=True)

In [ ]:
df.head()

In [ ]:
df['genre'].value_counts().head(20)

# could use weird genres as suplimental
# Want true genre label

## stretch: use nlp to classify genre of weird genres


# dataframe each movie as a row, colum should be all reviews of movie concat

# collab:
# # rmse 


# content filter:
# group by aggregate one long string for 

# top 200-500 words
# all features: pearson, co-sine, NN's, (sklearn.pairwise) 

# average rating could be used in meta return

In [ ]:
df1['details'].value_counts()

In [ ]:
df.to_csv('./data/meta_all_clean.csv', encoding='utf-8', index=False)

In [ ]:
#df['language'] = df['details'].apply(lambda x: x.get('Language:', ''))
#df['language'] = df['details'].apply(lambda x: x.get('Language:', '').split(' (')[0])
#df['language'] = df['language'].apply(lambda x: x.split(", "))
#df['language'] = df['language'].apply(lambda x: None if len(x)==0 or (len(x)==1 and x[0]=='') else x)

#df['subtitle'] = df['details'].apply(lambda x: x.get('Subtitles:', ''))
#df['subtitle'] = df['details'].apply(lambda x: x.get('Subtitles:', '').split(' (')[0])
#df['subtitle'] = df['subtitle'].apply(lambda x: x.split(", "))
#df['subtitle'] = df['subtitle'].apply(lambda x: None if len(x)==0 or (len(x)==1 and x[0]=='') else x)

#df['dub'] = df['details'].apply(lambda x: x.get('Dubbed:', ''))
#df['dub'] = df['details'].apply(lambda x: x.get('Dubbed:', '').split(' (')[0])
#df['dub'] = df['dub'].apply(lambda x: x.split(", "))
#df['dub'] = df['dub'].apply(lambda x: None if len(x)==0 or (len(x)==1 and x[0]=='') else x)

#df.head()